In [10]:
import pickle
import json
import pandas as pd

# Load the results

In [11]:
scores = []
with open('scores.pkl', 'rb') as f:
    scores = pickle.load(f)

In [12]:
column_names = "url;A;F;I;R;A1;F1;F2;F3;F4;I1;I2;I3;R1;R1.1;R1.2;R1.3;FAIR;FAIR%;FsF-F1-01D;FsF-F1-01D-1;FsF-F1-01D-2;FsF-F1-02D;FsF-F1-02D-1;FsF-F1-02D-2;FsF-F2-01M;FsF-F2-01M-1;FsF-F2-01M-2;FsF-F2-01M-3;FsF-F3-01M;FsF-F3-01M-1;FsF-F3-01M-2;FsF-F4-01M;FsF-F4-01M-1;FsF-F4-01M-2;FsF-A1-01M;FsF-A1-01M-1;FsF-A1-01M-3;FsF-A1-01M-2;FsF-A1-02M;FsF-A1-02M-1;FsF-A1-03D;FsF-A1-03D-1;FsF-I1-01M;FsF-I1-01M-1;FsF-I1-01M-2;FsF-I2-01M;FsF-I2-01M-1;FsF-I2-01M-2;FsF-I3-01M;FsF-I3-01M-1;FsF-I3-01M-2;FsF-R1-01MD;FsF-R1-01MD-1;FsF-R1-01MD-1a;FsF-R1-01MD-1b;FsF-R1-01MD-2;FsF-R1-01MD-2a;FsF-R1-01MD-2b;FsF-R1-01MD-2c;FsF-R1-01MD-3;FsF-R1-01MD-4;FsF-R1.1-01M;FsF-R1.1-01M-1;FsF-R1.1-01M-2;FsF-R1.2-01M;FsF-R1.2-01M-1;FsF-R1.2-01M-2;FsF-R1.3-01M;FsF-R1.3-01M-1;FsF-R1.3-01M-2;FsF-R1.3-01M-3;FsF-R1.3-02D;FsF-R1.3-02D-1;FsF-R1.3-02D-1a;FsF-R1.3-02D-1b;FsF-R1.3-02D-1c".split(";")

In [13]:
invenio_data = {}
enhanced_data = {}

for score in scores:
    invenio_row = []
    enhanced_row = []
    
    invenio_row.append(score['invenio_url'])
    enhanced_row.append(score['enhanced_url'])
    
    invenio_json = score['invenio_score']
    enhanced_json = score['enhanced_score']
    
    for k,v in invenio_json['summary']['score_earned'].items():
        invenio_row.append(v)
    for k,v in enhanced_json['summary']['score_earned'].items():
        enhanced_row.append(v)
        
    invenio_row.append(invenio_json['summary']['score_percent']['FAIR'])
    enhanced_row.append(enhanced_json['summary']['score_percent']['FAIR'])
        
    for metric in invenio_json['results']:
        invenio_row.append(metric['score']['earned'])
        for test, value in metric['metric_tests'].items():
            invenio_row.append(value['metric_test_score']['earned'])
            
    for metric in enhanced_json['results']:
        enhanced_row.append(metric['score']['earned'])
        for test, value in metric['metric_tests'].items():
            enhanced_row.append(value['metric_test_score']['earned'])
    
    invenio_data[score['record_id']] = invenio_row
    enhanced_data[score['record_id']] = enhanced_row

invenio_df = pd.DataFrame.from_dict(data=invenio_data, columns=column_names, orient='index')
enhanced_df = pd.DataFrame.from_dict(data=enhanced_data, columns=column_names, orient='index')


In [14]:
summary = {}
metrics = {}
tests = {}
for score in scores:
    invenio_json = score['invenio_score']
    for k,v in invenio_json['summary']['score_total'].items():
        summary[k] = v
        
    for metric in invenio_json['results']:
        metrics[metric['metric_identifier']] = metric['score']['total']
        for test, value in metric['metric_tests'].items():
            tests[test] = value['metric_test_score']['total']

In [15]:
# remove an outlier
invenio_df = invenio_df.drop(index="z8z7p-tgw13")
enhanced_df = enhanced_df.drop(index="z8z7p-tgw13")

-----

# Detailed report on the *FsF-R1-01MD-2* and *FsF-R1-01MD-3* FUJI tests
Below, we present parts of the FUJI logs that explain why the *FsF-R1-01MD-2* and *FsF-R1-01MD-3* tests failed for the metadata directly exported from InvenioRDM.

The lack of the `distribution` attribute, results in the `NO info about data service endpoint available in given metadata` error. Furthermore, FUJI finds wrong file sizes and mime types for certain records.

In [16]:
for idx in invenio_df[invenio_df['FsF-R1-01MD-3'] != tests['FsF-R1-01MD-3']].index:
    with open(f'invenio_scores/{idx}.json', 'r') as f:
        contents = json.load(f)
        for result in contents['results']:
            if result['metric_identifier'] == 'FsF-R1-01MD':
                print(f"------------- {idx} -------------")
                print(f'url: https://test.researchdata.tuwien.ac.at/records/{idx}')
                print(f'data: https://test.researchdata.tuwien.ac.at/api/records/{idx}/files')
                print(result['test_debug'][-3])
                print(result['test_debug'][-2])
                print(f"------------- ---- -------------")

------------- jcpn6-vnm19 -------------
url: https://test.researchdata.tuwien.ac.at/records/jcpn6-vnm19
data: https://test.researchdata.tuwien.ac.at/api/records/jcpn6-vnm19/files
------------- ---- -------------
------------- 05mkq-2xz41 -------------
url: https://test.researchdata.tuwien.ac.at/records/05mkq-2xz41
data: https://test.researchdata.tuwien.ac.at/api/records/05mkq-2xz41/files
INFO: Sucessfully verified content size from downloaded file -: (expected: 12.0, found: 12)
------------- ---- -------------
------------- 26sb2-d7635 -------------
url: https://test.researchdata.tuwien.ac.at/records/26sb2-d7635
data: https://test.researchdata.tuwien.ac.at/api/records/26sb2-d7635/files
INFO: NO info about data service endpoint available in given metadata for -: https://test.researchdata.tuwien.at/api/records/26sb2-d7635/files/testfile.txt/content
INFO: Sucessfully verified content type from downloaded file -: (expected: text/plain, found: via tika ['text/plain'] or via header text/plai

# Detailed report on the *FsF-I2-01M-2* FUJI test
Below, we present parts of the FUJI logs that explain why the *FsF-I2-01M-2* test failed for the metadata directly exported from InvenioRDM.

For the majority of the records, the problem was not including any annotations from outside the commonly used vocabularies.


In [17]:
for idx in invenio_df[invenio_df['FsF-I2-01M-2'] != tests['FsF-I2-01M-2']].index:
    with open(f'invenio_scores/{idx}.json', 'r') as f:
        contents = json.load(f)
        for result in contents['results']:
            if result['metric_identifier'] == 'FsF-I2-01M':
                print(f"------------- {idx} -------------")
                print(f'url: https://test.researchdata.tuwien.ac.at/records/{idx}')
                print(f'data: https://test.researchdata.tuwien.ac.at/api/records/{idx}/files')
                print(result['test_debug'][-1])
                print(f"------------- ---- -------------")

------------- yvh49-kvd12 -------------
url: https://test.researchdata.tuwien.ac.at/records/yvh49-kvd12
data: https://test.researchdata.tuwien.ac.at/api/records/yvh49-kvd12/files
------------- ---- -------------
------------- 3tf7q-e9t81 -------------
url: https://test.researchdata.tuwien.ac.at/records/3tf7q-e9t81
data: https://test.researchdata.tuwien.ac.at/api/records/3tf7q-e9t81/files
------------- ---- -------------
------------- jcpn6-vnm19 -------------
url: https://test.researchdata.tuwien.ac.at/records/jcpn6-vnm19
data: https://test.researchdata.tuwien.ac.at/api/records/jcpn6-vnm19/files
------------- ---- -------------
------------- 05mkq-2xz41 -------------
url: https://test.researchdata.tuwien.ac.at/records/05mkq-2xz41
data: https://test.researchdata.tuwien.ac.at/api/records/05mkq-2xz41/files
------------- ---- -------------
------------- 26sb2-d7635 -------------
url: https://test.researchdata.tuwien.ac.at/records/26sb2-d7635
data: https://test.researchdata.tuwien.ac.at/api

# Detailed report on the *FsF-R1.2-01M-2* FUJI test
Below, we present parts of the FUJI logs that explain why the *FsF-R1.2-01M-2* test failed for the metadata directly exported from InvenioRDM.
Reason: no annotations from the prov-o vocabulary found in the metadata.


In [18]:
for idx in invenio_df[invenio_df['FsF-R1.2-01M-2'] != tests['FsF-R1.2-01M-2']].index:
    with open(f'invenio_scores/{idx}.json', 'r') as f:
        contents = json.load(f)
        for result in contents['results']:
            if result['metric_identifier'] == 'FsF-R1.2-01M':
                print(f"------------- {idx} -------------")
                print(f'url: https://test.researchdata.tuwien.ac.at/records/{idx}')
                print(f'data: https://test.researchdata.tuwien.ac.at/api/records/{idx}/files')
                print(result['test_debug'][-1])
                print(f"------------- ---- -------------")

------------- yvh49-kvd12 -------------
url: https://test.researchdata.tuwien.ac.at/records/yvh49-kvd12
data: https://test.researchdata.tuwien.ac.at/api/records/yvh49-kvd12/files
------------- ---- -------------
------------- 3tf7q-e9t81 -------------
url: https://test.researchdata.tuwien.ac.at/records/3tf7q-e9t81
data: https://test.researchdata.tuwien.ac.at/api/records/3tf7q-e9t81/files
------------- ---- -------------
------------- jcpn6-vnm19 -------------
url: https://test.researchdata.tuwien.ac.at/records/jcpn6-vnm19
data: https://test.researchdata.tuwien.ac.at/api/records/jcpn6-vnm19/files
------------- ---- -------------
------------- 05mkq-2xz41 -------------
url: https://test.researchdata.tuwien.ac.at/records/05mkq-2xz41
data: https://test.researchdata.tuwien.ac.at/api/records/05mkq-2xz41/files
------------- ---- -------------
------------- 26sb2-d7635 -------------
url: https://test.researchdata.tuwien.ac.at/records/26sb2-d7635
data: https://test.researchdata.tuwien.ac.at/api

# Detailed logs about *FsF-R1-01MD-2* and *FsF-R1-01MD-3* FUJI tests
Below we present parts of FUJI logs that explain why the tests: *FsF-R1-01MD-2* *FsF-R1-01MD-3* failed, even for our improved metadata format. 

For the majority of the records, it is due to FUJIs failure to recognize a set of mime types exported from InvenioRDM. Those are: 
- `audio/x-wav`, 
- `application/pdf`, 
- `application/x-netcdf`, 
- `text/csv` and 
- `text/x-python`.

The presented logs also include information about file sizes. For records: `ecbjk-7ka62` and `apwsf-ejr45`, which contain empty files (`fileSize`: `0B`), FUJI fails to validate this information (which is included in the metadata!). That's why these two records fail the *FsF-R1-01MD-2* test.

In [19]:
for idx in enhanced_df[enhanced_df['FsF-R1-01MD-3'] != tests['FsF-R1-01MD-3']].index:
    with open(f'enhanced_scores/{idx}.json', 'r') as f:
        contents = json.load(f)
        for result in contents['results']:
            if result['metric_identifier'] == 'FsF-R1-01MD':
                print(f"------------- {idx} -------------")
                print(f'url: https://test.researchdata.tuwien.ac.at/records/{idx}')
                print(f'data: https://test.researchdata.tuwien.ac.at/api/records/{idx}/files')
                print(result['test_debug'][-3])
                print(result['test_debug'][-2])
                print(f"------------- ---- -------------")
                

------------- jcpn6-vnm19 -------------
url: https://test.researchdata.tuwien.ac.at/records/jcpn6-vnm19
data: https://test.researchdata.tuwien.ac.at/api/records/jcpn6-vnm19/files
INFO: Sucessfully verified content size from downloaded file -: (expected: 1792556.0, found: 1792556)
------------- ---- -------------
------------- 05mkq-2xz41 -------------
url: https://test.researchdata.tuwien.ac.at/records/05mkq-2xz41
data: https://test.researchdata.tuwien.ac.at/api/records/05mkq-2xz41/files
INFO: Sucessfully verified content size from downloaded file -: (expected: 12.0, found: 12)
------------- ---- -------------
------------- dc4zh-9ce78 -------------
url: https://test.researchdata.tuwien.ac.at/records/dc4zh-9ce78
data: https://test.researchdata.tuwien.ac.at/api/records/dc4zh-9ce78/files
INFO: Sucessfully verified content size from downloaded file -: (expected: 2318414.0, found: 2318414)
------------- ---- -------------
------------- pqb7y-mtf49 -------------
url: https://test.researchda